# Indexación de documentos mediante vectores de peso (TFxIDF) - Ejemplo con motor de búsqueda de documentos

In [7]:
# Para importar el módulo con las funciones necesarias
import sys
sys.path.append(r"D:\Repos Github\AnaliticaTextual-Ejemplos") # la carpeta donde está el módulo

import utils # el módulo -> utils.py 

In [20]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import es_core_news_sm  # procesador de lenguaje con segmentación de frases, tokenización, lematización, etiquetas POS y dependencias sintácticas. Carga el modelo en español de Spacy

In [9]:
# Con la función CrearQuery se prentende
    # Construir un vector TF-IDF para una consulta (query) usando el vocabulario entrenado previamente.
    # Si tengo un vocabulario como {"chile":0, "corea":1, "kpop":2, "idol":3, "fan":4} y el usario escribe "fan de kpop" -> La función generará un vector de la query: [0, 0, 1, 0, 1] * idf
    # bPara luego compararlo con otros documentos mediante coseno.

In [22]:
def crearQuery(terms,idf,vocabulario):
    query = np.zeros(len(vocabulario)) #Crea el vector de ceros del tamaño del vocabulario
    listaTerminos = utils.Tokenizar(utils.Lematizar(terms.lower())) #lematiza y tokenizar -> creando una lista de tokens → ["correr", "detrás", "de", "los", "idol"]. Tb me aseguro que está todo en minúsculas
    for t in listaTerminos:      
       if t in vocabulario: # vocabulario viene definido en la función  CrearVSM en utils.py-> vocabulario = modelo.vocabulary_
           indice = vocabulario[t]
           query[indice] = 1 #indice-> Posicón ; 1 -> valor que escribo en esta posicón. En este caso 1 = "existencia"
        
    if (np.count_nonzero(query) == 0): # Si el vector está vacío (ningún término válido), no tiene sentido devolverlo
        return None
    
    query = query * idf #Aplicar los pesos IDF (TF binario × IDF). idf viene de la fucnión CrearVSM en utils.py ->idf = modelo.idf         
    return query

In [11]:
# Esta función es un buscador: 
    # recibe una query vectorizada
    # la compara con cada documento del corpus
    # calcula la similitud
    # Devuelve la lista ordenada del más similar al menos similar
# Es el paso final del modelo vectorial TF-IDF

def RecuperarDocumentosRelevantes(query,modelo,doc_id):
  RelDocs = []
  for ind_doc in range(len(doc_id)): #doc_id viene de CrearCorpus en utils.py
    filename = doc_id[ind_doc]  
    similitud = 1 - cosine(query,modelo[ind_doc,:])
    RelDocs.append((similitud,filename))  
  return(sorted(RelDocs,reverse=True))

Síntesis:

distancia (cosine)	1 - distancia	interpretación

0.0	                      1.0	muy similar

0.3	                      0.7	medianamente

1.0	                      0.0	nada similar

In [13]:
def MostrarDocumentos(Docs): # se alimenta del resultado de RecuperarDocumentosRelevantes -> Docs = lista de tuplas (similitud, nombre_archivo).
    print("Lista de documentos relevantes a la query:\n")
    for (sim,d) in Docs:
        print("Doc: "+d+" ("+str(sim)+")\n") # + -> concatena

In [18]:
PATH = r"D:\Repos Github\AnaliticaTextual-Ejemplos\CORPUS\deportes"

nlp           = es_core_news_sm.load()
corpus,docsID = utils.CrearCorpus(PATH)
textos  = utils.PreProcesar(corpus)
utils.CrearVSM(textos,"mi_modelo")

(tfidf, idf, vocabulario) = utils.CargarModelo("mi_modelo")

In [23]:
print("*********************************************")
print("Bienvenido al buscador de documentos!")
print("*********************************************")

terms = input("Ingrese query: ")
vector_query = crearQuery(terms,idf,vocabulario)

if len(vector_query)==0:
    print("ERROR en vector de consulta, no se pueden recuperar documentos!..")
else:
    DocsRelevantes = RecuperarDocumentosRelevantes(vector_query,tfidf,docsID)
    MostrarDocumentos(DocsRelevantes)

*********************************************
Bienvenido al buscador de documentos!
*********************************************


Ingrese query:  jugador suizo


Lista de documentos relevantes a la query:

Doc: d9.txt (0.077986343624597)

Doc: d21.txt (0.06413267447149873)

Doc: d34.txt (0.04225895295397564)

Doc: d23.txt (0.038842974418654475)

Doc: d30.txt (0.03621130288623453)

Doc: d24.txt (0.0361293257964439)

Doc: d18.txt (0.033389305242030876)

Doc: d33.txt (0.03268121290887327)

Doc: d27.txt (0.030200074622632966)

Doc: d11.txt (0.025235341157737734)

Doc: d7.txt (0.02458539337536314)

Doc: d10.txt (0.016574196143898212)

Doc: d8.txt (0.0)

Doc: d6.txt (0.0)

Doc: d5.txt (0.0)

Doc: d4.txt (0.0)

Doc: d32.txt (0.0)

Doc: d31.txt (0.0)

Doc: d3.txt (0.0)

Doc: d29.txt (0.0)

Doc: d28.txt (0.0)

Doc: d26.txt (0.0)

Doc: d25.txt (0.0)

Doc: d22.txt (0.0)

Doc: d20.txt (0.0)

Doc: d2.txt (0.0)

Doc: d19.txt (0.0)

Doc: d17.txt (0.0)

Doc: d16.txt (0.0)

Doc: d15.txt (0.0)

Doc: d14.txt (0.0)

Doc: d13.txt (0.0)

Doc: d12.txt (0.0)

Doc: d1.txt (0.0)



Todo lo que se está haciendo en este notebook y .utils se puede ver así:

1) Preparar datos

CrearCorpus(path) → devuelve:

corpus = lista de textos

doc_id = lista de nombres de archivo

2) Crear o cargar el modelo vectorial

CrearVSM(textos, nombre_modelo, ...) → crea matriz TF/TF-IDF y la guarda con joblib

CargarModelo(nombre_modelo) → devuelve:

modelo (matriz documento–término)

idf

vocabulario

3) Preprocesar una query

crearQuery(terms, idf, vocabulario) → devuelve un vector TF-IDF para la consulta

4) Calcular similitudes y ordenar

RecuperarDocumentosRelevantes(query, modelo, doc_id) → lista de (similitud, filename)

Mostrar resultados

MostrarDocumentos(Docs) → imprime esa lista bonita

5) Resumen del flujo:

textos → corpus → VSM → query → similitudes → ranking → imprimir